<a href="https://colab.research.google.com/github/sowhyG/netology_BD_course/blob/master/task_final/fw_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [ ]:
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz

spark-2.4.5-bin-hadoop2.7/
spark-2.4.5-bin-hadoop2.7/licenses/
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-jtransforms.html
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-zstd.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-zstd-jni.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-xmlenc.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-vis.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-spire.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-sorttable.js.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-slf4j.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-scopt.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-scala.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-sbt-launch-lib.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-respond.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-reflectasm.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-pyrolite.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-py4j.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-protobuf.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-pmml-model

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [ ]:
from pyspark.sql.functions import when, col, mean, avg

###Загружаем данные

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

--2020-04-25 19:05:25--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  12.6MB/s    in 0.4s    

2020-04-25 19:05:26 (12.6 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]



In [ ]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-100k/allbut.pl       
replace ml-100k/mku.sh? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating

In [ ]:
schema = StructType([
    StructField('user_id', IntegerType(), True),
    StructField('item_id', IntegerType(), True),
    StructField('rating', IntegerType(), True),
    StructField('timestamp', IntegerType(), True)])
ratings = spark.read.csv('ml-100k/u.data', schema=schema, header=None, sep = '\t')
# ratings = spark.read.csv('ml-latest-small/ratings.csv', header=True, inferSchema=True)

In [ ]:
ratings.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [ ]:
# ratings.count()

In [ ]:
schema = StructType([
    StructField('movie_id', IntegerType(), True),
    StructField('movie_title', StringType(), True),
    StructField('release_date', StringType(), True),
    StructField('video_release_year', IntegerType(), True),
    StructField('IMDb_URL', StringType(), True),
    StructField('unknown', IntegerType(), True),
    StructField('Action', IntegerType(), True),
    StructField('Adventure', IntegerType(), True),
    StructField('Animation', IntegerType(), True),
    StructField('Childrens', IntegerType(), True),
    StructField('Comedy', IntegerType(), True),
    StructField('Crime', IntegerType(), True),
    StructField('Documentary', IntegerType(), True),
    StructField('Drama', IntegerType(), True),
    StructField('Fantasy', IntegerType(), True),
    StructField('Film-Noir', IntegerType(), True),
    StructField('Horror', IntegerType(), True),
    StructField('Musical', IntegerType(), True),
    StructField('Mystery', IntegerType(), True),
    StructField('Romance', IntegerType(), True),
    StructField('Sci-Fi', IntegerType(), True),
    StructField('Thriller', IntegerType(), True),
    StructField('War', IntegerType(), True),
    StructField('Western', IntegerType(), True)])
movies = spark.read.csv('ml-100k/u.item', schema=schema, header=None, sep = '|', encoding='ISO-8859-1')

In [ ]:
movies.show(5)

+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|      movie_title|release_date|video_release_year|            IMDb_URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|              null|

In [ ]:
# movies.count()

### Средние оценки для каждого фильма

In [ ]:
from pyspark.sql import functions as f
average_ratings = ratings.groupBy('item_id').agg(f.avg(ratings.rating).alias('avg'))
average_ratings.show()

+-------+------------------+
|item_id|               avg|
+-------+------------------+
|    496| 4.121212121212121|
|    471|3.6108597285067874|
|    463| 3.859154929577465|
|    148|          3.203125|
|   1342|               2.5|
|    833| 3.204081632653061|
|   1088| 2.230769230769231|
|   1591|3.1666666666666665|
|   1238|             3.125|
|   1580|               1.0|
|   1645|               4.0|
|    392|3.5441176470588234|
|    623| 2.923076923076923|
|    540| 2.511627906976744|
|    858|               1.0|
|    737| 2.983050847457627|
|    243|2.4393939393939394|
|   1025|2.9318181818181817|
|   1084| 3.857142857142857|
|   1127| 2.909090909090909|
+-------+------------------+
only showing top 20 rows



In [ ]:
# Список фильмов с средними значениями оценок, ранжированный по убыванию этого показателя:
movies_with_rating = average_ratings.join(movies, average_ratings.item_id == movies.movie_id, 'inner').select('movie_id', 'movie_title', 'avg')
movies_with_rating.orderBy('avg', ascending=False).show()

+--------+--------------------+-----------------+
|movie_id|         movie_title|              avg|
+--------+--------------------+-----------------+
|    1293|     Star Kid (1997)|              5.0|
|    1122|They Made Me a Cr...|              5.0|
|    1500|Santa with Muscle...|              5.0|
|    1467|Saint of Fort Was...|              5.0|
|     814|Great Day in Harl...|              5.0|
|    1201|Marlene Dietrich:...|              5.0|
|    1653|Entertaining Ange...|              5.0|
|    1536|Aiqing wansui (1994)|              5.0|
|    1189|  Prefontaine (1997)|              5.0|
|    1599|Someone Else's Am...|              5.0|
|    1449|Pather Panchali (...|            4.625|
|    1642|Some Mother's Son...|              4.5|
|    1594|      Everest (1998)|              4.5|
|    1398|         Anna (1996)|              4.5|
|     119|Maya Lin: A Stron...|              4.5|
|     408|Close Shave, A (1...|4.491071428571429|
|     318|Schindler's List ...|4.466442953020135|


In [ ]:
# movies_with_rating.count()

### Средние оценкии по жанрам фильмов

In [ ]:
movies_with_genres = ratings.join(movies, ratings.item_id == movies.movie_id, 'inner').select(
    'user_id', 'movie_id', 'rating', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
    'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
    )
movies_with_genres.show()

+-------+--------+------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|user_id|movie_id|rating|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+-------+--------+------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|    196|     242|     3|      0|     0|        0|        0|        0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|    186|     302|     3|      0|     0|        0|        0|        0|     0|    1|          0|    0|      0|        1|     0|      0|      1|      0|     0|       1|  0|      0|
|     22|     377|     1|      0|     0|        0|        0|        1|     1|    0|          0|    0|    

In [ ]:
movies_with_genres.count()

100000

In [ ]:
# Для жанра Action:
movies_with_genres.filter(movies_with_genres['Action'] == 1).agg({'rating': 'avg'}).show()
# movies_with_genres.where(movies_with_genres['Action'] == 1).count()

+-----------------+
|      avg(rating)|
+-----------------+
|3.480245417953027|
+-----------------+



In [ ]:
# movies_with_genres.filter(movies_with_genres['unknown'] == 1).agg({'rating': 'avg'}).show()
# movies_with_genres.filter(movies_with_genres['Adventure'] == 1).agg({'rating': 'avg'}).show()

In [ ]:
# Для всех жанров:
cols = [f.when(f.col(x).isin(0), None).otherwise(f.col(x) * f.col('rating')).alias(x)  for x in movies_with_genres.columns]
movie_genres_with_score = movies_with_genres.select(*cols)
movie_genres_with_score.show()

+-------+--------+------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+----+-------+
|user_id|movie_id|rating|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller| War|Western|
+-------+--------+------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+----+-------+
|    588|     726|     9|   null|  null|     null|     null|     null|     3| null|       null| null|   null|     null|  null|   null|   null|   null|  null|    null|null|   null|
|    558|     906|     9|   null|  null|     null|     null|     null|  null|    3|       null| null|   null|        3|  null|   null|      3|   null|  null|       3|null|   null|
|     22|     377|     1|   null|  null|     null|     null|        1|     1| null|       null| null

In [ ]:
movie_genres_with_score.agg(*(avg(col(c)).alias(c) for c in movies_with_genres.columns)).select(movies_with_genres.columns[3:]).show()

+-------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+----------------+-----------------+------------------+------------------+-----------------+------------------+
|unknown|           Action|        Adventure|         Animation|         Childrens|            Comedy|             Crime|       Documentary|             Drama|           Fantasy|         Film-Noir|            Horror|          Musical|         Mystery|          Romance|            Sci-Fi|          Thriller|              War|           Western|
+-------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+----------------+-----------------+------------------+------------------+---

### Рейтинги фильмов по количеству отзывов (или средним оценкам)

In [ ]:
movies_list = movies_with_genres.groupBy('movie_id').agg(f.count(movies_with_genres.rating).alias('count'), f.avg(movies_with_genres.rating).alias('avg'))
movies_list.show()

+--------+-----+------------------+
|movie_id|count|               avg|
+--------+-----+------------------+
|     496|  231| 4.121212121212121|
|     471|  221|3.6108597285067874|
|     463|   71| 3.859154929577465|
|     148|  128|          3.203125|
|    1342|    2|               2.5|
|     833|   49| 3.204081632653061|
|    1088|   13| 2.230769230769231|
|    1591|    6|3.1666666666666665|
|    1238|    8|             3.125|
|    1580|    1|               1.0|
|    1645|    1|               4.0|
|     392|   68|3.5441176470588234|
|     623|   39| 2.923076923076923|
|     540|   43| 2.511627906976744|
|     858|    3|               1.0|
|     737|   59| 2.983050847457627|
|     243|  132|2.4393939393939394|
|    1025|   44|2.9318181818181817|
|    1084|   21| 3.857142857142857|
|    1127|   11| 2.909090909090909|
+--------+-----+------------------+
only showing top 20 rows



In [ ]:
# movies_list.count()

In [ ]:
# Список фильмов с количеством и средними значениями оценок
movies_with_avg = movies_list.join(movies, on='movie_id').select('movie_id', 'movie_title', 'count', 'avg')

In [ ]:
# Топ-5 популярных фильмов по количеству оценок
movies_with_avg.orderBy('count', ascending=False).show(5)

+--------+--------------------+-----+------------------+
|movie_id|         movie_title|count|               avg|
+--------+--------------------+-----+------------------+
|      50|    Star Wars (1977)|  583|4.3584905660377355|
|     258|      Contact (1997)|  509|3.8035363457760316|
|     100|        Fargo (1996)|  508| 4.155511811023622|
|     181|Return of the Jed...|  507| 4.007889546351085|
|     294|    Liar Liar (1997)|  485| 3.156701030927835|
+--------+--------------------+-----+------------------+
only showing top 5 rows



In [ ]:
# Топ-5 непопулярных фильмов по количеству оценок
movies_with_avg.orderBy('count').show(5)

+--------+--------------------+-----+---+
|movie_id|         movie_title|count|avg|
+--------+--------------------+-----+---+
|    1339|Stefano Quantesto...|    1|1.0|
|    1507|Three Lives and O...|    1|3.0|
|    1580|     Liebelei (1933)|    1|1.0|
|    1460|    Sleepover (1995)|    1|3.0|
|    1645|Butcher Boy, The ...|    1|4.0|
+--------+--------------------+-----+---+
only showing top 5 rows

